# Humans and Flows
**In this tutorial you will:**
- learn how humans can be represented as Flows
- learn how Flows can request human input
- use a simple user interface to chat with Flows
- chat with your neighbor through Flows

There are many practical use cases of AI where we need to enable flexible interaction between humans and AI systems. For example, a customer support chatbot that is unable to help a customer should be able to request an internal human expert for help. A personal AI assistant booking a restaurant for a human might need to ask the human about dietary restrictions. In general, when an AI agent encounters a roadblock - it should be able to reach out to humans for help.

Furthermore, humans can be used to help AI systems with tasks that require complex reasoning. One such example is competitive programming - given a competitive programming problem, an LLM might struggle to solve it on it's own. However, we can include a human in the loop by having it provide a high-level plan on how to solve a given problem. We have implemented this particular example using aiFlows - learn more about it [here](https://huggingface.co/aiflows/CCFlows).

Depending on the workflow you are trying to automate, you might want to have a human "chip in" at certain places and under certain conditions. This is particularly important at the early stages of integrating AI into your business.

**In aiFlows, we view humans as tools that are simply wrapped with a Flow abstraction. This allows us to treat a human as any other regular Flow, enabling us to easily create complex interactions between humans and AI systems.**

In [1]:
%load_ext autoreload
%autoreload 2
import os, json, copy
from colink import CoLink
from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_thread, run_get_instance_worker_thread
from aiflows.backends.api_info import ApiInfo
from aiflows import flow_verse

## HumanFlow

A Flow designed to encapsulate a human will simply relay received messages to the human and then transmit the human's response back to the original sender. We can essentially think of a human as being the implementation of the Flow's ```run()``` method. We refer to such flow as a "HumanFlow". 

In this notebook we will demonstrate how to run a simple chat interaction between a human and a chabot using aiFlows. We will facilitate the interaction as a Composite Flow that has two subflows: a ChatFlow and a HumanFlow. 

## Connect to CoLink Server

To run this example, we will need a CoLink server running outside of the notebook because we will need to start the human UI in a separate process.

You can run the example by connecting to our official CoLink server or by starting a local CoLink server instance in a separate shell.
If you wish to run locally, please install the latest server release from here: https://github.com/CoLearn-Dev/colink-server-dev/releases

We will continue by connecting to our hosted CoLink server. If you don't have a colink user with our server, please do the following:

Run ```python generate_user.py```

This will generate your key pair and a signature of your intent to register with our server. The script will generate a file called user.txt which will contain your pubkey and signature. You should then provide this file to our server administrators - they will create a JWT for you, allowing you to connect to our CoLink server. Feel free to share this file with us through our [Discord](https://discord.gg/UbQ5JYtP)!


In [15]:
addr = "https://amld.colink-server.colearn.cloud"
jwt = ... # YOUR JWT
cl = CoLink(addr, jwt)
print(cl.get_user_id())

02e404b1aace07470c1dd18c25078274f6184e4dd4c741334b0b1d2e75168f9877


## ChatFlows UI

We have prepared a simple script that serves an AtomicFlow wrapper around a human, and starts a user interface through which the human can reply to messages directed at that flow, termed the HumanFlow. For practical use, you can integrate HumanFlows into your frontend applications by running a background CoLink worker process that receives messages sent to the HumanFlow and triggers an event in your application.

To serve the HumanFlow and start the UI:
- navigate to the /chatflows-ui folder
- add your JWT to the ui-config.yaml file
- run the application with ```streamlit run chatflows-ui.py```

While you are there, take a peek at the HumanUIFlow.yaml file which contains the flow config for the HumanFlow.

We have now served the HumanFlow as a singleton under the flow endpoint "User" (you can modify the endpoint in ui-config.yaml). We can now use the HumanFlow as a subflow of a larger Composite Flow, or we can directly interact with it by obtaining a proxy - let's do that first to ensure everything works as expected.

We will start a few dispatch workers (we'll also need them to run flows later). Notice that we are passing the API info when starting the workers. The workers will inject the API info when loading the Flow for execution - we do this for privacy reasons, so that the API info doesn't get stored in the CoLink server.

In [3]:
api_infos = [ApiInfo(backend_used="openai", api_key=os.getenv("OPENAI_API_KEY"))]
run_dispatch_worker_thread(cl, api_infos = api_infos)
run_dispatch_worker_thread(cl, api_infos = api_infos)

[2024-03-22 11:38:47,027][aiflows.workers.dispatch_worker:235][INFO] - Dispatch worker started in attached thread.
[2024-03-22 11:38:47,029][aiflows.workers.dispatch_worker:236][INFO] - dispatch_point: coflows_dispatch
[2024-03-22 11:38:47,051][aiflows.workers.dispatch_worker:235][INFO] - Dispatch worker started in attached thread.
[2024-03-22 11:38:47,053][aiflows.workers.dispatch_worker:236][INFO] - dispatch_point: coflows_dispatch
[2024-03-22 11:41:08,277][aiflows.workers.dispatch_worker:119][INFO] - 
~~~ Dispatch task ~~~
[2024-03-22 11:41:08,303][aiflows.workers.dispatch_worker:161][INFO] - flow_endpoint: InteractiveChat
[2024-03-22 11:41:08,304][aiflows.workers.dispatch_worker:162][INFO] - flow_id: 48e3b2c9-e0b6-42e9-a352-45300ada504f
[2024-03-22 11:41:08,305][aiflows.workers.dispatch_worker:163][INFO] - owner_id: local
[2024-03-22 11:41:08,305][aiflows.workers.dispatch_worker:164][INFO] - message_paths: ['push_tasks:e4ffbc27-490e-4720-b3a9-9534f23baeb6:msg']
[2024-03-22 11:41:08

In [4]:
user_flow = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="User",
)

input_data = {"id": 0, "api_output": "Human, are you there?"}
input_message = user_flow.package_input_message(input_data)

user_flow.get_reply_future(input_message).get_data() # blocks until human responds via UI

[2024-03-22 11:38:49,128][aiflows.utils.serve_utils:543][INFO] - Fetched singleton 5184fdd6-cddf-4326-b4f0-49633779785c


{'human_input': 'yes!! present!'}

Since CoLink allows us to invoke Flows served by other users, you can also invoke another user's HumanFlow! You just need to plug in their colink user ID when getting the flow instance. Just make sure that you are both running get_instance workers needed for getting/serving flow instances across users (note that running the UI automatically starts a get_instance worker in the background).

In [12]:
# if you don't have the UI running, uncomment and run the line below
# run_get_instance_worker_thread(cl)

[2024-03-22 11:43:09,055][aiflows.workers.get_instance_worker:164][INFO] - get_instances worker started in attached thread for user 0398851c0934a00b839a595fe06a50c3adeed560567a2f4fcecf987c77222d9751


In [13]:
user_flow = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="User",
    user_id=... # your neighbor's colink user ID
)

input_data = {"id": 0, "api_output": "Neighbor, are you there?"}
input_message = user_flow.package_input_message(input_data)

user_flow.get_reply_future(input_message).get_data() # blocks until human responds via UI

{'human_input': 'Hey there neighbor!!'}

## Orchestrating the interaction

Now let's try orchestrating a back and forth interaction between a ChatFlow and a HumanFlow. We start by fetching necessary Flows from the FlowVerse and serving them.

In [5]:
dependencies = [
    {"url": "aiflows/ChatFlowModule", "revision": "coflows"},
    {"url": "aiflows/ChatInteractiveFlowModule", "revision": "coflows"}
]
flow_verse.sync_dependencies(dependencies)

[2024-03-22 11:39:54,211][aiflows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /home/staverm/workspace/coflows-dev/aiflows/examples/mauro_examples/flow_modules...
[2024-03-22 11:39:54,354][aiflows.flow_verse.loading:608][INFO] - aiflows/ChatFlowModule:coflows already synced, skip
[2024-03-22 11:39:54,454][aiflows.flow_verse.loading:608][INFO] - aiflows/ChatInteractiveFlowModule:coflows already synced, skip
[2024-03-22 11:39:54,455][aiflows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing




['/home/staverm/workspace/coflows-dev/aiflows/examples/mauro_examples/flow_modules/aiflows/ChatFlowModule',
 '/home/staverm/workspace/coflows-dev/aiflows/examples/mauro_examples/flow_modules/aiflows/ChatInteractiveFlowModule',
 '/home/staverm/workspace/coflows-dev/aiflows/examples/mauro_examples/flow_modules/aiflows/HumanStandardInputFlowModule']

In [6]:
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow",
    flow_endpoint="Assistant",
)
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="flow_modules.aiflows.ChatInteractiveFlowModule.ChatHumanFlowModule",
    flow_endpoint="InteractiveChat",
)

[2024-03-22 11:39:58,246][aiflows.utils.serve_utils:116][INFO] - Started serving flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow at flows:Assistant.
[2024-03-22 11:39:58,248][aiflows.utils.serve_utils:117][INFO] - dispatch_point: coflows_dispatch
[2024-03-22 11:39:58,249][aiflows.utils.serve_utils:118][INFO] - parallel_dispatch: False
[2024-03-22 11:39:58,249][aiflows.utils.serve_utils:119][INFO] - singleton: False

[2024-03-22 11:39:58,340][aiflows.utils.serve_utils:116][INFO] - Started serving flow_modules.aiflows.ChatInteractiveFlowModule.ChatHumanFlowModule at flows:InteractiveChat.
[2024-03-22 11:39:58,341][aiflows.utils.serve_utils:117][INFO] - dispatch_point: coflows_dispatch
[2024-03-22 11:39:58,342][aiflows.utils.serve_utils:118][INFO] - parallel_dispatch: False
[2024-03-22 11:39:58,342][aiflows.utils.serve_utils:119][INFO] - singleton: False



True

The ChatHumanFlowModule is a simple composite flows that relays messages between it's two subflows (ChatAtomicFlow and HumanFlow), facilitating the "chat" interaction between them.

In [7]:
interactive_chat = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="InteractiveChat",
)

[2024-03-22 11:40:00,920][aiflows.flow_verse.loading:775][INFO] - [flow_modules.aiflows.ChatInteractiveFlowModule] started to sync flow module dependencies to /home/staverm/workspace/coflows-dev/aiflows/examples/mauro_examples/flow_modules...
[2024-03-22 11:40:01,065][aiflows.flow_verse.loading:608][INFO] - aiflows/ChatFlowModule:coflows already synced, skip
[2024-03-22 11:40:01,159][aiflows.flow_verse.loading:608][INFO] - aiflows/HumanStandardInputFlowModule:coflows already synced, skip
[2024-03-22 11:40:01,161][aiflows.flow_verse.loading:825][INFO] - [flow_modules.aiflows.ChatInteractiveFlowModule] finished syncing


[2024-03-22 11:40:01,339][aiflows.utils.serve_utils:336][INFO] - Mounted fd56e5c1-f1db-4824-bb1a-ac69147b54b4 at flows:Assistant:mounts:local:fd56e5c1-f1db-4824-bb1a-ac69147b54b4
[2024-03-22 11:40:01,427][aiflows.utils.serve_utils:543][INFO] - Fetched singleton 5184fdd6-cddf-4326-b4f0-49633779785c
[2024-03-22 11:40:01,514][aiflows.utils.serve_utils:336][INFO] - Mounted 4

We will kickstart the chat interaction by sending a message to the Composite Flow which will first relay this message to the Assistant, and then relay the Assistant's output message to our HumanFlow - this message should then get displayed in your UI. After receiving the message, you can respond to it via the UI and keep chatting with the Assistant. You can stop the chat orchestration by typing \<END>.

In [8]:
input_message = FlowMessage(
    data={"id": 0, "query": "I want to ask you a few questions"},
)
interactive_chat.send_message(input_message)